# Лабораторная работа №3

## Задание 1 . 
###### Провести классификацию найденного датасета, методами линеной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [2]:
data = pd.read_csv('database.csv',low_memory=False)
# Удаление ненужных столбцов
data = data.drop(columns=["Record ID", "Record Source"])
# Заполнение пропущенных значений
data = data.dropna()

###### Выбираем случайное подмножество данных, т.к. загруженный датасет содержит 638 454 записей которые будут обрабатываться слишком долго

In [3]:
subset_data = data.sample(n=11000, random_state=42)

subset_data.head() # Просмотр первых нескольких строк данных

,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,Crime Solved,...,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,CA03612,Victorville,Municipal Police,San Bernardino,California,2005,July,1,Murder or Manslaughter,No,...,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,TX10100,Harris County,Sheriff,Harris,Texas,2013,May,3,Murder or Manslaughter,No,...,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,AR06200,St. Francis,Sheriff,St. Francis,Arkansas,2001,June,1,Murder or Manslaughter,Yes,...,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,AZ01003,Tucson,Municipal Police,Pima,Arizona,1998,March,6,Murder or Manslaughter,No,...,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,WA02400,Okanogan,Sheriff,Okanogan,Washington,1999,August,1,Murder or Manslaughter,Yes,...,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


In [4]:
# Выбор признаков и целевой переменной
X = subset_data[['Victim Age', 'Victim Race', 'Victim Sex', 'Perpetrator Age', 'Perpetrator Race', 'Perpetrator Sex']]
Y = subset_data['Crime Solved'] 

###### Преобразование категориальных признаков в числовые

In [5]:
X = pd.get_dummies(X)

###### Разделение данных на обучающий и тестовый наборы

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###### Масштабирование данных
__*fit_transform*__ вычисляет среднее значение и стандартное отклонение для каждого признака на обучающем наборе данных

__*transform*__ применяет параметры масштабирования, вычисленные на обучающем наборе данных, к тестовому набору данных

In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Выбор гиперпараметров 
###### Кроссвалидация

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Создание экземпляра модели
model = LogisticRegression(max_iter=1000)

# Определение сетки значений параметра регуляризации C
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Создание объекта GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5)

# Обучение модели с кросс-валидацией
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]})

In [18]:
# Вывод результатов кросс-валидации
cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print("Mean Test Score:", mean_score, ", Parameters:", params)


Mean Test Score: 0.9986363636363634 , Parameters: {'C': 0.001}
Mean Test Score: 0.9993181818181818 , Parameters: {'C': 0.01}
Mean Test Score: 0.9993181818181818 , Parameters: {'C': 0.1}
Mean Test Score: 0.9995454545454546 , Parameters: {'C': 1}
Mean Test Score: 0.9995454545454546 , Parameters: {'C': 10}
Mean Test Score: 0.9995454545454546 , Parameters: {'C': 100}


In [17]:
# Вывод наилучших гиперпараметров
print("Best Parameters:", grid_search.best_params_)

# Вывод наилучшего среднего значения качества по кросс-валидации
print("Best Cross-Validation Score:", grid_search.best_score_)

Best Parameters: {'C': 1}
Best Cross-Validation Score: 0.9995454545454546


### Обучение и оценка модели с лучшими гиперпараметрами

In [21]:
# Создание экземпляра модели с лучшими гиперпараметрами
best_model = LogisticRegression(max_iter=1000, C=grid_search.best_params_['C'])

# Обучение модели на всем обучающем наборе данных
best_model.fit(X_train, Y_train)

# Предсказание на тестовом наборе данных
Y_pred = best_model.predict(X_test)

# Оценка производительности модели
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(Y_test, Y_pred))

Accuracy: 0.9995454545454545
Classification Report:
               precision    recall  f1-score   support

          No       1.00      1.00      1.00       650
         Yes       1.00      1.00      1.00      1550

    accuracy                           1.00      2200
   macro avg       1.00      1.00      1.00      2200
weighted avg       1.00      1.00      1.00      2200



###### Precision (точность): __TP / (TP + FP)__
Это показатель того, как много из предсказанных положительных случаев действительно являются положительными. 

###### Recall (полнота): __TP / (TP + FN)__
Это показатель того, как много реальных положительных случаев модель предсказала правильно.

###### F1-score: __2 * (precision * recall) / (precision + recall)__
Это гармоническое среднее между precision и recall. Это показатель общей производительности классификатора. 

###### Support: 
Это количество фактических вхождений каждого класса в тестовом наборе данных
___
###### Accuracy (точность) __(TP + TN) / (TP + TN + FP + FN)__ 
Это процент правильно классифицированных случаев. 

- TP - количество верно предсказанных положительных случаев 
- TN - количество верно предсказанных отрицательных случаев 
- FP - количество ложно предсказанных положительных случаев 
- FN - количество ложно предсказанных отрицательных случаев 

###### Macro avg: 
Средние значения __precision__, __recall__ и __f1-score__ по всем классам без учета их дисбаланса.

###### Weighted avg: 
Средние значения __precision__, __recall__ и __f1-score__ по всем классам, учитывая их дисбаланс по количеству.